In [1]:
import time
import sys
sys.path.append('../')
#import librairies.dagfeaturingfx 
#from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
#from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait() 

Importing Librairies...


###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

%%time

_t1 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)

TICKER_LIST = ['GBP/CAD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
_period = 'm5'
df_all = pd.read_csv(x.replace('/','')+'_'+_period+'_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
df_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
df_all['Lindex'] = pd.to_datetime(df_all['Date'] + ' ' + df_all['Time'])
df_all.set_index(pd.to_datetime(df_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
df_all = df_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
df_all['WE'] = np.where(((df_all.index.weekday == 5) | (df_all.index.weekday == 6)),None,df_all.index.weekday)
df_all = df_all.dropna()
df_all = df_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
df_all['Open'] = (df_all['OpenBid'] + df_all['OpenAsk']) / 2
df_all['High'] = (df_all['HighBid'] + df_all['HighAsk']) / 2
df_all['Low'] = (df_all['LowBid'] + df_all['LowAsk']) / 2
df_all['Close'] = (df_all['CloseBid'] + df_all['CloseAsk']) / 2

hourly_all = pd.read_csv(x.replace('/','')+'_H1_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
hourly_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
hourly_all['Lindex'] = pd.to_datetime(hourly_all['Date'] + ' ' + hourly_all['Time'])
hourly_all.set_index(pd.to_datetime(hourly_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
hourly_all = hourly_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
hourly_all['WE'] = np.where(((hourly_all.index.weekday == 5) | (hourly_all.index.weekday == 6)),None,hourly_all.index.weekday)
hourly_all = hourly_all.dropna()
hourly_all = hourly_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
hourly_all['Open'] = (hourly_all['OpenBid'] + hourly_all['OpenAsk']) / 2
hourly_all['High'] = (hourly_all['HighBid'] + hourly_all['HighAsk']) / 2
hourly_all['Low'] = (hourly_all['LowBid'] + hourly_all['LowAsk']) / 2
hourly_all['Close'] = (hourly_all['CloseBid'] + hourly_all['CloseAsk']) / 2

engine.say("Processing featuring of dataframes, daily and intra-day")
engine.runAndWait()
##### Récupération des data pour tous les tickers sur la période demandée en intraday => df_all

engine.say("Raw data are loaded in memory")
engine.runAndWait()


df_all = timerange1D(df_all)
hourly_all = timerange1D(hourly_all)
daily_all = get_daily(hourly_all,TICKER_LIST)

engine.say("Daily up to date")
engine.runAndWait()

daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("Weekly up to date")
engine.runAndWait()

##### On calcule l'ADR sur le daily
daily_all = adr(daily_all,_window=14)
engine.say("ADR computed")
engine.runAndWait()

##### On récupère l'ADR qui a été calculé en daily (daily_all) pour le mettre dans la base intraday df_all
df_all = getadr(daily_all,df_all,TICKER_LIST)
engine.say("ADR get in daily")
engine.runAndWait()

df_all = adrhnl(daily_all,df_all,TICKER_LIST)

##### Calcul d'une SMA 200 sur df_all
df_all = sma(df_all=df_all,_window=200)

##### Calcul des bollinger sur df_all
df_all = bollinger(df_all,_slow=20)

##### Calcul du stochastic slow. Par défaut les paramètres sont 5 et 3.
df_all = slowstochastic(df_all,TICKER_LIST)

df_all = ema(df_all,21,TICKER_LIST)

df_all = ema(df_all,8,TICKER_LIST)

weekly_all = pivot(weekly_all,TICKER_LIST)

df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)

df_all = atr(df_all,TICKER_LIST,14)

df_all = rvi(df_all,TICKER_LIST,_window=14)

df_all = sbgamma(df_all,TICKER_LIST)

df_all = onhisma(df_all,TICKER_LIST,_window=5)
df_all = onlosma(df_all,TICKER_LIST,_window=5)

df_all = onhisma(df_all,TICKER_LIST,_window=21)
df_all = onlosma(df_all,TICKER_LIST,_window=21)

df_all = onhisma(df_all,TICKER_LIST,_window=34)
df_all = onlosma(df_all,TICKER_LIST,_window=34)

df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')

df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')

engine.say("The job is done")
engine.runAndWait()


engine.say("AAll the bases are saved")
engine.runAndWait()

_t2 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
print((_t2 - _t1))

%%time
joblib.dump(df_all,x.replace('/','')+'_DF_ALL')
joblib.dump(hourly_all,x.replace('/','')+'_HOURLY_ALL')
joblib.dump(weekly_all,x.replace('/','')+'_WEEKLY_ALL')
joblib.dump(daily_all,x.replace('/','')+'_DAILY_ALL')

In [4]:
%%time
TICKER_LIST = ['GBP/NZD']
x = TICKER_LIST[0]
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
hourly_all = joblib.load(x.replace('/','')+'_HOURLY_ALL')
weekly_all = joblib.load(x.replace('/','')+'_WEEKLY_ALL')
daily_all = joblib.load(x.replace('/','')+'_DAILY_ALL')


CPU times: user 484 ms, sys: 444 ms, total: 927 ms
Wall time: 963 ms


In [5]:
%%time

def featuring(df_all):
    """[Entrer la df préparée avec les bons indictaurs. Renvoie une nouvelle df qu'avec les features + Symbol + Date + Signal]

    Args:
        df_all ([dataframe]): [Mettre la df qui doit être featurée.]
    """   
    features = pd.DataFrame()
    features = pd.DataFrame(index=df_all.index)
    features['Symbol'] = df_all['Symbol']
    features['Date'] = df_all['Date']
    features['FEMA_21'] = df_all['Close'] - df_all['EMA_21']
    features['FEMA_8'] = df_all['Close'] - df_all['EMA_8']
    features['FADRLo'] = df_all['Close'] - df_all['ADR_Low']
    features['FADRHi'] = df_all['Close'] - df_all['ADR_High']
    features['FRVI40'] = df_all['RVI'] - 40
    features['FRVI60'] = df_all['RVI'] - 60
    features['FONLOSMA5'] = df_all['Low'] - df_all['ONLOSMA_5']
    features['FONHISMA5'] = df_all['High'] - df_all['ONHISMA_5']
    features['FONLOSMA21'] = df_all['Low'] - df_all['ONLOSMA_21']
    features['FONHISMA21'] = df_all['High'] - df_all['ONHISMA_21']
    features['FONLOSMA34'] = df_all['Low'] - df_all['ONLOSMA_34']
    features['FONHISMA34'] = df_all['High'] - df_all['ONHISMA_34']
    features['FSBGAMMA'] = df_all['SB_Gamma']
    features['FOPENWEEKLY'] = df_all['Close'] - df_all['Open_weekly']
    features['FHIGHWEEKLY'] = df_all['Close'] - df_all['High_weekly']
    features['FLOWWEEKLY'] = df_all['Close'] - df_all['Low_weekly']
    features['FCLOSEWEEKLY'] = df_all['Close'] - df_all['Close_weekly']
    features['FOPENDAILY'] = df_all['Close'] - df_all['Open_daily']
    features['FHIGHDAILY'] = df_all['Close'] - df_all['High_daily']
    features['FLOWDAILY'] = df_all['Close'] - df_all['Low_daily']
    features['FCLOSEDAILY'] = df_all['Close'] - df_all['Close_daily']
    features['FOPENHOURLY'] = df_all['Close'] - df_all['Open_daily']
    features['FHIGHHOURLY'] = df_all['Close'] - df_all['High_daily']
    features['FLOWHOURLY'] = df_all['Close'] - df_all['Low_daily']
    features['FCLOSEHOURLY'] = df_all['Close'] - df_all['Close_daily']
    features['FSMA200'] = df_all['Close'] - df_all['SMA_200']
    features['FBOLUP20'] = df_all['Close'] - df_all['UpperBand']
    features['FBOLLOW20'] = df_all['Close'] - df_all['LowerBand']
    features['FPP'] = df_all['Close'] - df_all['PP']
    features['FS38'] = df_all['Close'] - df_all['S38']
    features['FS62'] = df_all['Close'] - df_all['S62']
    features['FS100'] = df_all['Close'] - df_all['S100']
    features['FS138'] = df_all['Close'] - df_all['S138']
    features['FS162'] = df_all['Close'] - df_all['S162']
    features['FS200'] = df_all['Close'] - df_all['S200']
    features['FR38'] = df_all['Close'] - df_all['R38']
    features['FR62'] = df_all['Close'] - df_all['R62']
    features['FR100'] = df_all['Close'] - df_all['R100']
    features['FR138'] = df_all['Close'] - df_all['R138']
    features['FR162'] = df_all['Close'] - df_all['R162']
    features['FR200'] = df_all['Close'] - df_all['R200']
    features['SBATR'] = (df_all['Close'] - df_all['Open']) / df_all['ATR_14']
    features['Signal'] = df_all['Signal']
    return(features)



def stochastic(df):
        """[Determine the stochastic strategy based on _window=5. Enter dataframe with column slow_K5 and slow_D5. Return dataframe with column Signal.]

        Args:
            df ([dataframe]): [Must been already computed with slow_D5 and slow_K5 column]
        """    
        ##### CONDITIONS LONG
        _condition_1 = (df.slow_K5 <= 20) & (df.slow_K5.shift(1) <= df.slow_D5.shift(1)) & (df.slow_K5 >= df.slow_D5)

        ##### CONDITIONS SHORT
        _condition_1_bar = (df.slow_K5 >= 80) & (df.slow_K5.shift(1) >= df.slow_D5.shift(1)) & (df.slow_K5 <= df.slow_D5)

        ##### 1 condition
        df['Signal'] = np.where(_condition_1,1,np.where(_condition_1_bar,-1,0))
        return(df) 

####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TRACKER = joblib.load('TRACKER')
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape,daily_all.shape,weekly_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2021-03-15' # stop the test there. After that, it is kept for oos
_last = '2021-03-24' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 1
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 10000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only

#features = featuring(df_all)
#features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
#features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
#features = features[features.Symbol==x.replace('/','')]

# And drop the nan
#features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

# Proceed an MaxAbsScaler on features
#features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
#features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

_model = joblib.load('Save_'+x.replace('/','')+'_m5.sav')


df_all_oos = df_all[(df_all.Date >= _stop)&(df_all.Date <= _last)].dropna()

#df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]

features_oos = featuring(df_all_oos)

features_max = joblib.load(x.replace('/','')+'_MAX.dagmax')

for label in features_oos.drop(['Date','Symbol','Signal'],axis=1).columns:
                    features_oos[label] = features_oos[label] / features_max[label][0]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal'],axis=1))

df_all_oos['Verif_raw_sig'] = df_all_oos.Signal
df_all_oos['Signal'] = np.where((df_all_oos.Verif_raw_sig==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Verif_raw_sig==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

print(x)
_year_bottom = _stop
_year_top = _last

##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(814131, 59) (2901, 7) (584, 19)
GBP/NZD
GBP/NZD
Librairies imported

Début des opérations horodatée à 2021-04-08 12:26:43.909506

Chargement de la nouvelle base


 Le rate du ticker GBP/NZD est à  1.3886960144424387 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 1798/1798 [00:00<00:00, 36870.75it/s]
 2021-03-15 16:15:00 
Position 1 bought at 1.9297 (verification liste 1.9297 )
 2021-03-15 21:10:00 
1 positions (SL) closed at 1.92673  pnl -29.69999999999917 
nombre de candles en position : 59
Equity : 199970.3
 2021-03-22 22:35:00 
Position 1 sold at 1.94173 (verification liste 1.94173 )
 2021-03-22 23:10:00 
1 position (SL) closed at 1.94504  pnl -33.10000000000146 
nombre de candles en position : 7
Equity : 199937.19999999998
 2021-03-23 03:35:00 
Position 1 sold at 1.95159 (verification liste 1.95159 )
 2021-03-23 05:25:00 
1 position (SL) closed at 1.95387  pnl -22.800000000000598 
nombre de candles en position : 22
Equity : 199914.4
 2021-03-23 08:

In [17]:
verif = joblib.load('GBPNZD_m5')
verif = verif.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
# Calculate the mid prices
verif['Open'] = (verif.OpenAsk + verif.OpenBid)/2
verif['High'] = (verif.HighAsk + verif.HighBid)/2
verif['Low'] = (verif.LowAsk + verif.LowBid)/2
verif['Close'] = (verif.CloseAsk + verif.CloseBid)/2
verif['Symbol'] = x.replace('/','')
verif['Date'] = verif.index
verif['Date'] = pd.to_datetime(verif['Date'].dt.strftime(date_format='%Y-%m-%d'))

In [18]:
verif = verif[(verif.Date >= _stop)&(verif.Date <= _last)].dropna()
verif = stochastic(verif)
features_oos = featuring(verif)

features_max = joblib.load(x.replace('/','')+'_MAX.dagmax')

for label in features_oos.drop(['Date','Symbol','Signal'],axis=1).columns:
                    features_oos[label] = features_oos[label] / features_max[label][0]

#features_oos = scaling(features_oos,scaler)
#features_oos = quantile(features_oos,quantile_transform)

verif['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal'],axis=1))

verif['Verif_raw_sig'] = verif.Signal
verif['Signal'] = np.where((verif.Signal==1)&(verif.Valid==1),1,np.where((verif.Signal==-1)&(verif.Valid==1),-1,0))
verif = verif.dropna()
print(x)
_year_bottom = _stop
_year_top = _last
feature_api = features_oos
##### Backtest Over Night
FINAL_TRACKER = bt(verif,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

GBP/NZD
Librairies imported

Début des opérations horodatée à 2021-03-23 10:11:16.109609

Chargement de la nouvelle base


 Le rate du ticker GBP/NZD est à  1.3886960144424387 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 397/397 [00:00<00:00, 45690.34it/s]
 2021-03-22 22:35:00 
Position 1 sold at 1.94173 (verification liste 1.94173 )
 2021-03-22 23:10:00 
1 position (SL) closed at 1.94504  pnl -33.10000000000146 
nombre de candles en position : 7
Equity : 199966.9
 2021-03-23 08:05:00 
Position 1 sold at 1.95879 (verification liste 1.95879 )
 2021-03-23 08:30:00 
1 position (TG) closed at 1.95485  pnl 39.400000000000546 
nombre de candles en position : 5
Equity : 200006.3
 For ticker  GBP/NZD 
 
Total Profit & Loss : $  8.75 . En  2   transactions.
 
Winners Number : 1 
 
Loosers number : 1 
BT's execution time 0:00:06.233987
 GBP/NZD  results 
 Tested Period 2021-03-21  à 2021-03-24 
 Total Number of trades 2 
Started Cash : 200000
P&L  in currency: 

In [19]:
verif1 = joblib.load('GBPCAD_m5')
verif1 = verif1.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
# Calculate the mid prices
verif1['Open'] = (verif1.OpenAsk + verif1.OpenBid)/2
verif1['High'] = (verif1.HighAsk + verif1.HighBid)/2
verif1['Low'] = (verif1.LowAsk + verif1.LowBid)/2
verif1['Close'] = (verif1.CloseAsk + verif1.CloseBid)/2
verif1['Symbol'] = x.replace('/','')
verif1['Date'] = verif1.index
verif1['Date'] = pd.to_datetime(verif1['Date'].dt.strftime(date_format='%Y-%m-%d'))
verif1 = verif1[(verif1.Date >= _stop)&(verif1.Date <= _last)].dropna()
verif1 = stochastic(verif1)
verif1 = verif1.drop(verif1.sample(50).index)
features_oos1 = featuring(verif1)

features_oos1 = featuring(verif1)

features_oos1 = scaling(features_oos1,scaler)
features_oos1 = quantile(features_oos1,quantile_transform)

verif1['Valid'] = _model.predict(features_oos1.drop(['Date','Symbol','Signal'],axis=1))

verif1['Verif_raw_sig'] = verif1.Signal
verif1['Signal'] = np.where((verif1.Signal==1)&(verif1.Valid==1),1,np.where((verif1.Signal==-1)&(verif1.Valid==1),-1,0))
verif1 = verif1.dropna()
print(x)
_year_bottom = _stop
_year_top = _last
feature_api1 = features_oos1
##### Backtest Over Night
FINAL_TRACKER = bt(verif1,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

ValueError: a must be greater than 0 unless no samples are taken

In [6]:
verif[(verif.index>='2021-02-26 19:40:00')&(verif.index<='2021-02-26 19:45:00')][['slow_K5','slow_D5','Signal','Valid','Verif_raw_sig']]

,slow_K5,slow_D5,Signal,Valid,Verif_raw_sig
2021-02-26 19:40:00,8.542570,17.646010,0,1,0
2021-02-26 19:45:00,13.157169,13.026805,1,1,1


In [7]:
df_all_oos[(df_all_oos.index>='2021-02-26 19:40:00')&(df_all_oos.index<='2021-02-26 19:45:00')][['slow_K5','slow_D5','Signal','Valid','Verif_raw_sig']]

,slow_K5,slow_D5,Signal,Valid,Verif_raw_sig
Lindex,,,,,
2021-02-26 19:40:00,8.542570,17.646010,0,1,0
2021-02-26 19:45:00,13.157169,13.026805,1,1,1


In [8]:
_model.predict(feature_api.drop(['Date','Symbol','Signal'],axis=1)).sum()

1664

In [9]:
_model.predict(feature_hdd.drop(['Date','Symbol','Signal'],axis=1)).sum()

1683

In [10]:
(quantile(scaling(featuring(verif[(verif.index>='2021-02-26 19:40:00')&(verif.index<='2021-02-26 19:45:00')]).drop(['Date','Symbol'],axis=1),scaler),quantile_transform)-quantile(scaling(featuring(df_all_oos[(df_all_oos.index>='2021-02-26 19:40:00')&(df_all_oos.index<='2021-02-26 19:45:00')]).drop(['Date','Symbol'],axis=1),scaler),quantile_transform)).T

,2021-02-26 19:40:00,2021-02-26 19:45:00
FEMA_21,0.000000e+00,0.000000e+00
FEMA_8,0.000000e+00,0.000000e+00
FADRLo,0.000000e+00,0.000000e+00
FADRHi,0.000000e+00,0.000000e+00
FRVI40,0.000000e+00,0.000000e+00
FRVI60,0.000000e+00,0.000000e+00
FONLOSMA5,0.000000e+00,0.000000e+00
FONHISMA5,0.000000e+00,0.000000e+00
FONLOSMA21,0.000000e+00,0.000000e+00
FONHISMA21,0.000000e+00,0.000000e+00


In [11]:
df_all_oos[df_all_oos.Verif_raw_sig!=0]

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig
Lindex,,,,,,,,,,,,,,,,,,,,,
2021-01-04 00:10:00,1.73966,1.73991,1.73953,1.73979,1.74024,1.74045,1.74009,1.74036,GBPCAD,1.739950,...,1.74131,1.718890,1.722470,1.740735,1.74131,1.724660,1.734575,0,0,1
2021-01-04 02:00:00,1.73843,1.73881,1.73833,1.73873,1.73888,1.73925,1.73878,1.73920,GBPCAD,1.738655,...,1.74131,1.718890,1.722470,1.740735,1.74131,1.724660,1.734575,0,0,1
2021-01-04 04:35:00,1.73742,1.73777,1.73713,1.73773,1.73785,1.73820,1.73757,1.73809,GBPCAD,1.737635,...,1.74131,1.718890,1.722470,1.740735,1.74131,1.724660,1.734575,0,0,1
2021-01-04 05:50:00,1.73869,1.73869,1.73816,1.73824,1.73913,1.73913,1.73854,1.73866,GBPCAD,1.738910,...,1.74131,1.718890,1.722470,1.740735,1.74131,1.724660,1.734575,0,0,-1
2021-01-04 09:20:00,1.73615,1.73633,1.73450,1.73459,1.73652,1.73667,1.73482,1.73492,GBPCAD,1.736335,...,1.74131,1.718890,1.722470,1.740735,1.74131,1.724660,1.734575,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01 16:55:00,1.76428,1.76434,1.76326,1.76331,1.76457,1.76466,1.76357,1.76363,GBPCAD,1.764425,...,1.77793,1.760045,1.760765,1.774350,1.77793,1.760045,1.760765,0,0,1
2021-03-01 18:00:00,1.76231,1.76240,1.76170,1.76187,1.76264,1.76273,1.76204,1.76222,GBPCAD,1.762475,...,1.77793,1.760045,1.760765,1.774350,1.77793,1.760045,1.760765,0,0,1
2021-03-01 19:20:00,1.76163,1.76183,1.76146,1.76173,1.76195,1.76216,1.76178,1.76204,GBPCAD,1.761790,...,1.77793,1.760045,1.760765,1.774350,1.77793,1.760045,1.760765,0,0,1


In [12]:
df_all_oos.iloc[186:189,:].T

Lindex,2021-01-04 15:30:00,2021-01-04 15:35:00,2021-01-04 15:40:00
OpenBid,1.72744,1.72761,1.72909
HighBid,1.72811,1.7293,1.72909
LowBid,1.72692,1.72758,1.7272
CloseBid,1.72761,1.72909,1.72734
OpenAsk,1.72783,1.72803,1.72948
HighAsk,1.72846,1.72968,1.72948
LowAsk,1.72734,1.72797,1.7276
CloseAsk,1.72803,1.72948,1.72772
Symbol,GBPCAD,GBPCAD,GBPCAD
Open,1.72764,1.72782,1.72928


In [13]:
df_all[df_all.index=='2021-02-26 12:50:00'].T

Lindex,2021-02-26 12:50:00
OpenBid,1.76516
HighBid,1.7659
LowBid,1.76421
CloseBid,1.76452
OpenAsk,1.76553
HighAsk,1.76622
LowAsk,1.76457
CloseAsk,1.7649
Symbol,GBPCAD
Open,1.76534


In [14]:
df_all[(df_all.Date >= _stop)&(df_all.Date <= _last)].dropna()[df_all[(df_all.Date >= _stop)&(df_all.Date <= _last)].dropna().index=='2021-02-26 12:50:00'].T

Lindex,2021-02-26 12:50:00
OpenBid,1.76516
HighBid,1.7659
LowBid,1.76421
CloseBid,1.76452
OpenAsk,1.76553
HighAsk,1.76622
LowAsk,1.76457
CloseAsk,1.7649
Symbol,GBPCAD
Open,1.76534


In [15]:
verif[verif.index=='2021-02-26 15:35:00'].T

,2021-02-26 15:35:00
Close,1.76928
CloseAsk,1.76947
CloseBid,1.7691
High,1.76967
HighAsk,1.76985
...,...
Low_daily,1.75524
Close_daily,1.77572
Signal,0
Valid,0


In [16]:
df_all_oos[df_all_oos.duplicated()]

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig
Lindex,,,,,,,,,,,,,,,,,,,,,


In [17]:
verif[verif.duplicated()]

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig


In [18]:
verif.shape,features_oos.shape

((11667, 61), (11667, 45))

In [19]:
verif

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig
2021-01-04 00:00:00,1.739300,1.73954,1.73906,1.741180,1.74142,1.74094,1.739240,1.73949,1.73899,1.740735,...,1.741310,1.718890,1.722470,1.740735,1.741310,1.724660,1.734575,0,0,0
2021-01-04 00:05:00,1.739950,1.74024,1.73966,1.740125,1.74041,1.73984,1.739090,1.73935,1.73883,1.739300,...,1.741310,1.718890,1.722470,1.740735,1.741310,1.724660,1.734575,0,0,0
2021-01-04 00:10:00,1.740075,1.74036,1.73979,1.740180,1.74045,1.73991,1.739810,1.74009,1.73953,1.739950,...,1.741310,1.718890,1.722470,1.740735,1.741310,1.724660,1.734575,0,0,1
2021-01-04 00:15:00,1.740240,1.74048,1.74000,1.740360,1.74061,1.74011,1.739875,1.74012,1.73963,1.740075,...,1.741310,1.718890,1.722470,1.740735,1.741310,1.724660,1.734575,0,0,0
2021-01-04 00:20:00,1.740205,1.74045,1.73996,1.740555,1.74078,1.74033,1.740055,1.74030,1.73981,1.740240,...,1.741310,1.718890,1.722470,1.740735,1.741310,1.724660,1.734575,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01 23:35:00,1.761110,1.76129,1.76093,1.761170,1.76135,1.76099,1.760835,1.76101,1.76066,1.760940,...,1.777015,1.756635,1.760905,1.774330,1.777015,1.760345,1.761075,0,0,0
2021-03-01 23:40:00,1.760740,1.76092,1.76056,1.761145,1.76132,1.76097,1.760585,1.76077,1.76040,1.761115,...,1.777015,1.756635,1.760905,1.774330,1.777015,1.760345,1.761075,0,0,0
2021-03-01 23:45:00,1.760805,1.76097,1.76064,1.760865,1.76105,1.76068,1.760655,1.76083,1.76048,1.760730,...,1.777015,1.756635,1.760905,1.774330,1.777015,1.760345,1.761075,0,1,0
2021-03-01 23:50:00,1.760890,1.76106,1.76072,1.761000,1.76117,1.76083,1.760565,1.76073,1.76040,1.760795,...,1.777015,1.756635,1.760905,1.774330,1.777015,1.760345,1.761075,0,0,0


In [20]:
features_oos

,Symbol,Date,FEMA_21,FEMA_8,FADRLo,FADRHi,FRVI40,FRVI60,FONLOSMA5,FONHISMA5,...,FS162,FS200,FR38,FR62,FR100,FR138,FR162,FR200,SBATR,Signal
2021-01-04 00:00:00,GBPCAD,2021-01-04,0.005814,0.001160,0.974584,0.001098,0.373871,0.373871,0.000954,0.006232,...,0.032359,0.262763,0.445423,0.500324,0.575269,0.624827,0.271276,0.671004,0.009292,0
2021-01-04 00:05:00,GBPCAD,2021-01-04,0.018402,0.008245,0.989349,0.025789,0.521733,0.521733,0.001624,0.001287,...,0.033009,0.279086,0.485071,0.539560,0.604181,0.643590,0.285793,0.690913,0.881114,0
2021-01-04 00:10:00,GBPCAD,2021-01-04,0.031251,0.025257,0.999990,0.030434,0.656353,0.656353,0.039463,0.008328,...,0.033134,0.282045,0.493370,0.547857,0.611361,0.646931,0.287603,0.694135,0.604697,1
2021-01-04 00:15:00,GBPCAD,2021-01-04,0.052005,0.071659,0.999961,0.035455,0.757850,0.757850,0.316316,0.086425,...,0.033299,0.286768,0.506348,0.559325,0.620120,0.652382,0.290541,0.699180,0.638641,0
2021-01-04 00:20:00,GBPCAD,2021-01-04,0.060834,0.100153,0.999515,0.034072,0.828027,0.828027,0.881882,0.602603,...,0.033264,0.285285,0.504340,0.556837,0.618619,0.651469,0.289859,0.698345,0.472484,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01 23:35:00,GBPCAD,2021-03-01,0.633320,0.699232,0.018084,0.469469,0.423826,0.423826,0.550050,0.550050,...,0.043031,0.816327,0.013025,0.002601,0.002601,0.002601,0.688931,0.002601,0.750977,0
2021-03-01 23:40:00,GBPCAD,2021-03-01,0.393179,0.345095,0.013364,0.411023,0.345782,0.345782,0.301301,0.535536,...,0.042661,0.804745,0.008337,0.001063,0.001063,0.001063,0.672673,0.001063,0.082629,0
2021-03-01 23:45:00,GBPCAD,2021-03-01,0.439861,0.429908,0.014114,0.420643,0.278466,0.278466,0.405405,0.276276,...,0.042726,0.806341,0.009372,0.001479,0.001479,0.001479,0.675477,0.001479,0.625098,0
2021-03-01 23:50:00,GBPCAD,2021-03-01,0.500231,0.527337,0.015285,0.435435,0.218216,0.218216,0.345345,0.449950,...,0.042811,0.809074,0.010689,0.002011,0.002011,0.002011,0.679778,0.002011,0.656540,0


In [21]:
verif.dropna().shape, df_all_oos.dropna().shape

((11667, 61), (11589, 59))

In [22]:
verif.index[0],verif.index[-1],df_all_oos.index[0],df_all_oos.index[-1]

(Timestamp('2021-01-04 00:00:00'),
 Timestamp('2021-03-01 23:55:00'),
 Timestamp('2021-01-04 00:00:00'),
 Timestamp('2021-03-01 23:55:00'))

In [23]:
len(verif) - len(df_all_oos)

78

In [24]:
verif['datation'] = verif.index
verif['duplikindex'] = np.where((verif.datation==verif.datation.shift(1))|(verif.datation==verif.datation.shift(-1)),1,0)

In [25]:
verif[verif['duplikindex'] == 1]

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig,datation,duplikindex


In [26]:
df_all_oos['datation'] = df_all_oos.index
df_all_oos['duplikindex'] = np.where((df_all_oos.datation==df_all_oos.datation.shift(1))|(df_all_oos.datation==df_all_oos.datation.shift(-1)),1,0)
df_all_oos[df_all_oos['duplikindex'] == 1]

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig,datation,duplikindex
Lindex,,,,,,,,,,,,,,,,,,,,,


In [27]:
%%time
VERIF = []
for i in tqdm(range(len(verif))):
    if verif.datation[i] not in df_all_oos.datation.to_list():
        VERIF.append(verif.datation[i])
len(VERIF)

100%|██████████| 11667/11667 [03:17<00:00, 59.19it/s]CPU times: user 3min 11s, sys: 2.35 s, total: 3min 14s
Wall time: 3min 17s



105

In [28]:
VERIF

[Timestamp('2021-01-04 21:40:00'),
 Timestamp('2021-01-05 23:05:00'),
 Timestamp('2021-01-20 07:05:00'),
 Timestamp('2021-01-21 08:10:00'),
 Timestamp('2021-01-21 08:20:00'),
 Timestamp('2021-01-25 09:10:00'),
 Timestamp('2021-01-25 09:20:00'),
 Timestamp('2021-01-25 09:40:00'),
 Timestamp('2021-01-25 09:45:00'),
 Timestamp('2021-01-26 10:00:00'),
 Timestamp('2021-01-26 10:05:00'),
 Timestamp('2021-01-28 12:05:00'),
 Timestamp('2021-02-01 12:20:00'),
 Timestamp('2021-02-01 12:50:00'),
 Timestamp('2021-02-03 14:00:00'),
 Timestamp('2021-02-04 15:35:00'),
 Timestamp('2021-02-04 15:40:00'),
 Timestamp('2021-02-04 16:10:00'),
 Timestamp('2021-02-04 16:20:00'),
 Timestamp('2021-02-08 15:05:00'),
 Timestamp('2021-02-08 15:20:00'),
 Timestamp('2021-02-10 17:00:00'),
 Timestamp('2021-02-10 17:05:00'),
 Timestamp('2021-02-15 18:15:00'),
 Timestamp('2021-02-15 18:35:00'),
 Timestamp('2021-02-21 19:30:00'),
 Timestamp('2021-02-21 19:35:00'),
 Timestamp('2021-02-21 19:40:00'),
 Timestamp('2021-02-

In [29]:
%%time
FIREV = []
for i in tqdm(range(len(df_all_oos))):
    if df_all_oos.datation[i] not in verif.datation.to_list():
        FIREV.append(df_all_oos.datation[i])
len(FIREV)

100%|██████████| 11589/11589 [03:29<00:00, 55.36it/s]CPU times: user 3min 24s, sys: 2.53 s, total: 3min 26s
Wall time: 3min 29s



27

In [30]:
FIREV

[Timestamp('2021-01-04 03:45:00'),
 Timestamp('2021-01-04 04:00:00'),
 Timestamp('2021-01-08 06:30:00'),
 Timestamp('2021-01-12 09:00:00'),
 Timestamp('2021-01-14 11:05:00'),
 Timestamp('2021-01-14 11:20:00'),
 Timestamp('2021-01-14 11:30:00'),
 Timestamp('2021-01-19 13:00:00'),
 Timestamp('2021-01-20 14:05:00'),
 Timestamp('2021-01-20 14:20:00'),
 Timestamp('2021-01-21 14:45:00'),
 Timestamp('2021-01-21 15:00:00'),
 Timestamp('2021-01-25 16:20:00'),
 Timestamp('2021-01-26 16:35:00'),
 Timestamp('2021-01-26 16:45:00'),
 Timestamp('2021-01-26 17:05:00'),
 Timestamp('2021-01-28 18:45:00'),
 Timestamp('2021-01-28 19:05:00'),
 Timestamp('2021-01-28 19:20:00'),
 Timestamp('2021-02-17 23:40:00'),
 Timestamp('2021-02-18 00:10:00'),
 Timestamp('2021-02-18 02:10:00'),
 Timestamp('2021-02-18 03:00:00'),
 Timestamp('2021-02-18 04:15:00'),
 Timestamp('2021-02-18 04:30:00'),
 Timestamp('2021-02-18 04:45:00'),
 Timestamp('2021-02-18 05:05:00')]

In [31]:
105 - 78

27

In [32]:
verif[verif.index=='2021-02-18 05:05:00']

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,Valid,Verif_raw_sig,datation,duplikindex


In [34]:
%%time
feature_hdd['datation'] = feature_hdd.index
feature_api['datation'] = feature_api.index
GOLO = []
for i in tqdm(range(len(feature_hdd))):
    if feature_hdd.datation[i] not in feature_api.datation.to_list():
        GOLO.append(feature_hdd.datation[i])
print(len(GOLO))
GOLO

100%|██████████| 11589/11589 [04:55<00:00, 39.26it/s]27
CPU times: user 4min 47s, sys: 3.79 s, total: 4min 50s
Wall time: 4min 55s



[Timestamp('2021-01-04 03:45:00'),
 Timestamp('2021-01-04 04:00:00'),
 Timestamp('2021-01-08 06:30:00'),
 Timestamp('2021-01-12 09:00:00'),
 Timestamp('2021-01-14 11:05:00'),
 Timestamp('2021-01-14 11:20:00'),
 Timestamp('2021-01-14 11:30:00'),
 Timestamp('2021-01-19 13:00:00'),
 Timestamp('2021-01-20 14:05:00'),
 Timestamp('2021-01-20 14:20:00'),
 Timestamp('2021-01-21 14:45:00'),
 Timestamp('2021-01-21 15:00:00'),
 Timestamp('2021-01-25 16:20:00'),
 Timestamp('2021-01-26 16:35:00'),
 Timestamp('2021-01-26 16:45:00'),
 Timestamp('2021-01-26 17:05:00'),
 Timestamp('2021-01-28 18:45:00'),
 Timestamp('2021-01-28 19:05:00'),
 Timestamp('2021-01-28 19:20:00'),
 Timestamp('2021-02-17 23:40:00'),
 Timestamp('2021-02-18 00:10:00'),
 Timestamp('2021-02-18 02:10:00'),
 Timestamp('2021-02-18 03:00:00'),
 Timestamp('2021-02-18 04:15:00'),
 Timestamp('2021-02-18 04:30:00'),
 Timestamp('2021-02-18 04:45:00'),
 Timestamp('2021-02-18 05:05:00')]

In [35]:
%%time
feature_hdd['datation'] = feature_hdd.index
feature_api['datation'] = feature_api.index
OLGO = []
for i in tqdm(range(len(feature_api))):
    if feature_api.datation[i] not in feature_hdd.datation.to_list():
        OLGO.append(feature_api.datation[i])
print(len(OLGO))
OLGO

100%|██████████| 11667/11667 [03:49<00:00, 50.93it/s]105
CPU times: user 3min 42s, sys: 2.15 s, total: 3min 44s
Wall time: 3min 49s



[Timestamp('2021-01-04 21:40:00'),
 Timestamp('2021-01-05 23:05:00'),
 Timestamp('2021-01-20 07:05:00'),
 Timestamp('2021-01-21 08:10:00'),
 Timestamp('2021-01-21 08:20:00'),
 Timestamp('2021-01-25 09:10:00'),
 Timestamp('2021-01-25 09:20:00'),
 Timestamp('2021-01-25 09:40:00'),
 Timestamp('2021-01-25 09:45:00'),
 Timestamp('2021-01-26 10:00:00'),
 Timestamp('2021-01-26 10:05:00'),
 Timestamp('2021-01-28 12:05:00'),
 Timestamp('2021-02-01 12:20:00'),
 Timestamp('2021-02-01 12:50:00'),
 Timestamp('2021-02-03 14:00:00'),
 Timestamp('2021-02-04 15:35:00'),
 Timestamp('2021-02-04 15:40:00'),
 Timestamp('2021-02-04 16:10:00'),
 Timestamp('2021-02-04 16:20:00'),
 Timestamp('2021-02-08 15:05:00'),
 Timestamp('2021-02-08 15:20:00'),
 Timestamp('2021-02-10 17:00:00'),
 Timestamp('2021-02-10 17:05:00'),
 Timestamp('2021-02-15 18:15:00'),
 Timestamp('2021-02-15 18:35:00'),
 Timestamp('2021-02-21 19:30:00'),
 Timestamp('2021-02-21 19:35:00'),
 Timestamp('2021-02-21 19:40:00'),
 Timestamp('2021-02-

In [38]:
%%time
setb = set(OLGO)
print(len([x for x in GOLO if not x in setb]))
[x for x in GOLO if not x in setb]

27
CPU times: user 273 µs, sys: 80 µs, total: 353 µs
Wall time: 322 µs


[Timestamp('2021-01-04 03:45:00'),
 Timestamp('2021-01-04 04:00:00'),
 Timestamp('2021-01-08 06:30:00'),
 Timestamp('2021-01-12 09:00:00'),
 Timestamp('2021-01-14 11:05:00'),
 Timestamp('2021-01-14 11:20:00'),
 Timestamp('2021-01-14 11:30:00'),
 Timestamp('2021-01-19 13:00:00'),
 Timestamp('2021-01-20 14:05:00'),
 Timestamp('2021-01-20 14:20:00'),
 Timestamp('2021-01-21 14:45:00'),
 Timestamp('2021-01-21 15:00:00'),
 Timestamp('2021-01-25 16:20:00'),
 Timestamp('2021-01-26 16:35:00'),
 Timestamp('2021-01-26 16:45:00'),
 Timestamp('2021-01-26 17:05:00'),
 Timestamp('2021-01-28 18:45:00'),
 Timestamp('2021-01-28 19:05:00'),
 Timestamp('2021-01-28 19:20:00'),
 Timestamp('2021-02-17 23:40:00'),
 Timestamp('2021-02-18 00:10:00'),
 Timestamp('2021-02-18 02:10:00'),
 Timestamp('2021-02-18 03:00:00'),
 Timestamp('2021-02-18 04:15:00'),
 Timestamp('2021-02-18 04:30:00'),
 Timestamp('2021-02-18 04:45:00'),
 Timestamp('2021-02-18 05:05:00')]

In [39]:
%%time
seta = set(GOLO)
print(len([x for x in OLGO if not x in seta]))
[x for x in OLGO if not x in seta]

105
CPU times: user 268 µs, sys: 105 µs, total: 373 µs
Wall time: 444 µs


[Timestamp('2021-01-04 21:40:00'),
 Timestamp('2021-01-05 23:05:00'),
 Timestamp('2021-01-20 07:05:00'),
 Timestamp('2021-01-21 08:10:00'),
 Timestamp('2021-01-21 08:20:00'),
 Timestamp('2021-01-25 09:10:00'),
 Timestamp('2021-01-25 09:20:00'),
 Timestamp('2021-01-25 09:40:00'),
 Timestamp('2021-01-25 09:45:00'),
 Timestamp('2021-01-26 10:00:00'),
 Timestamp('2021-01-26 10:05:00'),
 Timestamp('2021-01-28 12:05:00'),
 Timestamp('2021-02-01 12:20:00'),
 Timestamp('2021-02-01 12:50:00'),
 Timestamp('2021-02-03 14:00:00'),
 Timestamp('2021-02-04 15:35:00'),
 Timestamp('2021-02-04 15:40:00'),
 Timestamp('2021-02-04 16:10:00'),
 Timestamp('2021-02-04 16:20:00'),
 Timestamp('2021-02-08 15:05:00'),
 Timestamp('2021-02-08 15:20:00'),
 Timestamp('2021-02-10 17:00:00'),
 Timestamp('2021-02-10 17:05:00'),
 Timestamp('2021-02-15 18:15:00'),
 Timestamp('2021-02-15 18:35:00'),
 Timestamp('2021-02-21 19:30:00'),
 Timestamp('2021-02-21 19:35:00'),
 Timestamp('2021-02-21 19:40:00'),
 Timestamp('2021-02-

In [41]:
%%time
seta = set(VERIF)
print(len([x for x in OLGO if not x in seta]))
[x for x in OLGO if not x in seta]

0
CPU times: user 415 µs, sys: 115 µs, total: 530 µs
Wall time: 480 µs


[]

In [42]:
%%time
seta = set(FIREV)
print(len([x for x in GOLO if not x in seta]))
[x for x in GOLO if not x in seta]

0
CPU times: user 335 µs, sys: 106 µs, total: 441 µs
Wall time: 387 µs


[]

In [60]:
feature_hdd.iloc[-1:,:].append(feature_api.iloc[-1:,:]).T

,2021-03-01 23:55:00,2021-03-01 23:55:00
Symbol,GBPCAD,GBPCAD
Date,2021-03-01 00:00:00,2021-03-01 00:00:00
FEMA_21,0.419323,0.419609
FEMA_8,0.400959,0.405329
FADRLo,0.0124261,0.0136476
FADRHi,0.376598,0.414384
FRVI40,0.235521,0.170997
FRVI60,0.235521,0.170997
FONLOSMA5,0.486486,0.486486
FONHISMA5,0.575075,0.574074


In [73]:
%%time
_condition = (abs(features_oos.datation.dt.minute - features_oos.datation.shift(1).dt.minute) !=5) | (abs(features_oos.datation.dt.minute - features_oos.datation.shift(-1).dt.minute) != 5)
features_oos['TimeJump'] = np.where(_condition,1,0)

CPU times: user 5.78 ms, sys: 2.78 ms, total: 8.56 ms
Wall time: 7.15 ms


In [77]:
features_oos[features_oos.TimeJump==1].shape,features_oos.shape

((1997, 47), (11667, 47))

In [78]:
features_oos[features_oos.TimeJump==1]

,Symbol,Date,FEMA_21,FEMA_8,FADRLo,FADRHi,FRVI40,FRVI60,FONLOSMA5,FONHISMA5,...,FR38,FR62,FR100,FR138,FR162,FR200,SBATR,Signal,datation,TimeJump
2021-01-04 00:00:00,GBPCAD,2021-01-04,0.005814,0.001160,0.974584,0.001098,0.373871,0.373871,0.000954,0.006232,...,0.445423,0.500324,0.575269,0.624827,0.271276,0.671004,0.009292,0,2021-01-04 00:00:00,1
2021-01-04 00:55:00,GBPCAD,2021-01-04,0.472621,0.824719,0.999486,0.012608,0.376620,0.376620,0.841174,0.795796,...,0.553981,0.597898,0.644083,0.675633,0.301279,0.716717,0.699715,0,2021-01-04 00:55:00,1
2021-01-04 01:00:00,GBPCAD,2021-01-04,0.242433,0.426442,0.996338,0.002114,0.302169,0.302169,0.575576,0.551552,...,0.525597,0.577018,0.632633,0.660438,0.294822,0.705706,0.134626,0,2021-01-04 01:00:00,1
2021-01-04 01:55:00,GBPCAD,2021-01-04,0.052739,0.060136,0.942097,0.000728,0.730834,0.730834,0.109109,0.094094,...,0.407863,0.458541,0.537538,0.599762,0.250929,0.649650,0.311175,0,2021-01-04 01:55:00,1
2021-01-04 02:00:00,GBPCAD,2021-01-04,0.099414,0.198343,0.961154,0.012316,0.628635,0.628635,0.158158,0.135135,...,0.427380,0.479901,0.558559,0.613520,0.260906,0.659358,0.820569,1,2021-01-04 02:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01 21:55:00,GBPCAD,2021-03-01,0.272608,0.527031,0.015199,0.424424,0.696329,0.696329,0.400400,0.776777,...,0.010614,0.001991,0.001991,0.001991,0.679507,0.001991,0.717329,0,2021-03-01 21:55:00,1
2021-03-01 22:00:00,GBPCAD,2021-03-01,0.303128,0.551792,0.015799,0.430114,0.608967,0.608967,0.685686,0.681682,...,0.011093,0.002091,0.002091,0.002091,0.681251,0.002091,0.415437,0,2021-03-01 22:00:00,1
2021-03-01 22:55:00,GBPCAD,2021-03-01,0.592430,0.781158,0.016644,0.448830,0.173590,0.173590,0.672673,0.853353,...,0.011703,0.002265,0.002265,0.002265,0.684058,0.002265,0.907375,0,2021-03-01 22:55:00,1
2021-03-01 23:00:00,GBPCAD,2021-03-01,0.645965,0.794913,0.017937,0.466719,0.275048,0.275048,0.366337,0.914305,...,0.012868,0.002560,0.002560,0.002560,0.688518,0.002560,0.653917,0,2021-03-01 23:00:00,1


In [106]:
%%time
df_all['datation'] = df_all.index
_condition = (abs(df_all.datation - df_all.datation.shift(1)) != dt.timedelta(minutes=5)) & (df_all.datation.dt.day == df_all.datation.shift(1).dt.day)
df_all['TimeJump'] = np.where(_condition,1,0)
df_all[(df_all.TimeJump==1)&(df_all.datation.dt.hour>=8)&(df_all.datation.dt.hour<=22)]

CPU times: user 118 ms, sys: 1.99 ms, total: 120 ms
Wall time: 111 ms


,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,datation,TimeJump
Lindex,,,,,,,,,,,,,,,,,,,,,
2010-01-22 16:50:00,1.70186,1.70186,1.69996,1.70024,1.70278,1.70278,1.70083,1.70087,GBPCAD,1.702320,...,1.709475,1.672170,1.702500,1.702180,1.706030,1.696230,1.702500,0,2010-01-22 16:50:00,1
2010-01-22 17:45:00,1.70194,1.70218,1.70170,1.70204,1.70213,1.70310,1.70213,1.70296,GBPCAD,1.702035,...,1.709475,1.672170,1.702500,1.702180,1.706030,1.696230,1.702500,0,2010-01-22 17:45:00,1
2010-02-11 22:45:00,1.64739,1.64749,1.64729,1.64729,1.64846,1.64856,1.64829,1.64851,GBPCAD,1.647925,...,1.680450,1.640195,1.648045,1.657935,1.658240,1.644910,1.649230,0,2010-02-11 22:45:00,1
2010-02-15 20:45:00,1.64229,1.64239,1.64199,1.64209,1.64381,1.64381,1.64298,1.64351,GBPCAD,1.643050,...,1.650900,1.606320,1.607945,1.646475,1.648670,1.639575,1.644220,0,2010-02-15 20:45:00,1
2010-02-15 22:30:00,1.64234,1.64269,1.64234,1.64269,1.64409,1.64448,1.64409,1.64441,GBPCAD,1.643215,...,1.650900,1.606320,1.607945,1.646475,1.648670,1.639575,1.644220,0,2010-02-15 22:30:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-10 17:10:00,1.75653,1.75670,1.75615,1.75646,1.75685,1.75700,1.75647,1.75678,GBPCAD,1.756690,...,1.761570,1.747590,1.758275,1.754395,1.759440,1.752955,1.756145,0,2021-02-10 17:10:00,1
2021-02-15 18:20:00,1.75693,1.75734,1.75641,1.75707,1.75772,1.75802,1.75743,1.75794,GBPCAD,1.757325,...,1.775145,1.756585,1.766960,1.759880,1.762815,1.756585,1.758190,0,2021-02-15 18:20:00,1
2021-02-15 18:40:00,1.75695,1.75710,1.75681,1.75700,1.75772,1.75786,1.75759,1.75775,GBPCAD,1.757335,...,1.775145,1.756585,1.766960,1.759880,1.762815,1.756585,1.758190,0,2021-02-15 18:40:00,1


In [107]:
len(df_all[df_all.TimeJump==1])/len(df_all)*100

0.1431665730735765

In [98]:
df_all[df_all.index >= '2010-01-11 00:00:00']

,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Symbol,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,datation,TimeJump
Lindex,,,,,,,,,,,,,,,,,,,,,
2010-01-11 00:00:00,1.65312,1.65389,1.65217,1.65222,1.65525,1.65555,1.65342,1.65370,GBPCAD,1.654185,...,1.68972,1.649120,1.673725,1.654185,1.670615,1.649120,1.666010,0,2010-01-11 00:00:00,1
2010-01-11 00:05:00,1.65222,1.65377,1.65114,1.65324,1.65370,1.65490,1.65289,1.65480,GBPCAD,1.652960,...,1.68972,1.649120,1.673725,1.654185,1.670615,1.649120,1.666010,0,2010-01-11 00:05:00,0
2010-01-11 00:10:00,1.65324,1.65397,1.65308,1.65329,1.65480,1.65503,1.65444,1.65473,GBPCAD,1.654020,...,1.68972,1.649120,1.673725,1.654185,1.670615,1.649120,1.666010,0,2010-01-11 00:10:00,0
2010-01-11 00:20:00,1.65329,1.65329,1.65254,1.65270,1.65473,1.65473,1.65371,1.65397,GBPCAD,1.654010,...,1.68972,1.649120,1.673725,1.654185,1.670615,1.649120,1.666010,0,2010-01-11 00:20:00,1
2010-01-11 00:25:00,1.65270,1.65329,1.65254,1.65329,1.65397,1.65428,1.65397,1.65408,GBPCAD,1.653335,...,1.68972,1.649120,1.673725,1.654185,1.670615,1.649120,1.666010,0,2010-01-11 00:25:00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-01 23:35:00,1.76076,1.76099,1.76066,1.76093,1.76111,1.76135,1.76101,1.76129,GBPCAD,1.760935,...,1.77793,1.760045,1.760765,1.774350,1.777930,1.760045,1.760765,0,2021-03-01 23:35:00,0
2021-03-01 23:40:00,1.76093,1.76097,1.76040,1.76056,1.76129,1.76132,1.76077,1.76092,GBPCAD,1.761110,...,1.77793,1.760045,1.760765,1.774350,1.777930,1.760045,1.760765,0,2021-03-01 23:40:00,0
2021-03-01 23:45:00,1.76056,1.76068,1.76048,1.76064,1.76092,1.76105,1.76083,1.76097,GBPCAD,1.760740,...,1.77793,1.760045,1.760765,1.774350,1.777930,1.760045,1.760765,0,2021-03-01 23:45:00,0


In [2]:
%%time
toto = pd.read_csv('GBPCAD_m5_BidAndAsk.csv')
toto.index = pd.to_datetime(toto.Date+' '+toto.Time)
toto['datation'] = toto.index
_condition = (abs(toto.datation - toto.datation.shift(1)) != dt.timedelta(minutes=5)) & (toto.datation.dt.day == toto.datation.shift(1).dt.day)
toto['TimeJump'] = np.where(_condition,1,0)
toto[(toto.TimeJump==1)&(toto.datation.dt.hour>=8)&(toto.datation.dt.hour<=22)]

CPU times: user 1min 39s, sys: 1.05 s, total: 1min 40s
Wall time: 1min 45s


,Date,Time,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Total Ticks,datation,TimeJump
2010-01-17 22:15:00,01-17-2010,22:15:00,1.67512,1.67629,1.67512,1.67626,1.67687,1.67779,1.67687,1.67776,17,2010-01-17 22:15:00,1
2010-01-22 16:50:00,01-22-2010,16:50:00,1.70186,1.70186,1.69996,1.70024,1.70278,1.70278,1.70083,1.70087,91,2010-01-22 16:50:00,1
2010-01-22 17:45:00,01-22-2010,17:45:00,1.70194,1.70218,1.70170,1.70204,1.70213,1.70310,1.70213,1.70296,64,2010-01-22 17:45:00,1
2010-01-31 22:20:00,01-31-2010,22:20:00,1.70419,1.70456,1.70412,1.70423,1.71005,1.71005,1.70616,1.70616,11,2010-01-31 22:20:00,1
2010-02-07 22:15:00,02-07-2010,22:15:00,1.66372,1.66632,1.66372,1.66632,1.66962,1.66962,1.66705,1.66799,13,2010-02-07 22:15:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-02-25 22:05:00,02-25-2021,22:05:00,1.76470,1.76531,1.76470,1.76473,1.76798,1.76839,1.76797,1.76798,5,2021-02-25 22:05:00,1
2021-02-25 22:15:00,02-25-2021,22:15:00,1.76473,1.76493,1.76421,1.76485,1.76798,1.76810,1.76640,1.76808,45,2021-02-25 22:15:00,1
2021-02-28 21:05:00,02-28-2021,21:05:00,1.77231,1.77231,1.77075,1.77084,1.77514,1.77514,1.77403,1.77406,2,2021-02-28 21:05:00,1
2021-02-28 21:30:00,02-28-2021,21:30:00,1.77146,1.77164,1.77108,1.77108,1.77455,1.77479,1.77423,1.77423,10,2021-02-28 21:30:00,1


In [3]:
len(toto[toto.TimeJump==1])/len(toto)*100

0.23808668367285837

In [128]:
toto[(toto.index>='2010-01-22 16:40:00')].head(60)

,Date,Time,OpenBid,HighBid,LowBid,CloseBid,OpenAsk,HighAsk,LowAsk,CloseAsk,Total Ticks,datation,TimeJump
2010-01-22 16:40:00,01-22-2010,16:40:00,1.70252,1.70252,1.70136,1.70186,1.70327,1.70327,1.70228,1.70278,46,2010-01-22 16:40:00,0
2010-01-22 16:50:00,01-22-2010,16:50:00,1.70186,1.70186,1.69996,1.70024,1.70278,1.70278,1.70083,1.70087,91,2010-01-22 16:50:00,1
2010-01-22 16:55:00,01-22-2010,16:55:00,1.70024,1.70139,1.70010,1.70121,1.70087,1.70231,1.70087,1.70213,134,2010-01-22 16:55:00,0
2010-01-22 17:00:00,01-22-2010,17:00:00,1.70121,1.70199,1.70121,1.70192,1.70213,1.70213,1.70213,1.70213,5,2010-01-22 17:00:00,0
2010-01-22 17:05:00,01-22-2010,17:05:00,1.70192,1.70203,1.70191,1.70194,1.70213,1.70213,1.70213,1.70213,7,2010-01-22 17:05:00,0
2010-01-22 17:45:00,01-22-2010,17:45:00,1.70194,1.70218,1.70170,1.70204,1.70213,1.70310,1.70213,1.70296,64,2010-01-22 17:45:00,1
2010-01-24 22:00:00,01-24-2010,22:00:00,1.70204,1.70313,1.70204,1.70261,1.70296,1.70520,1.70296,1.70476,6,2010-01-24 22:00:00,0
2010-01-24 22:05:00,01-24-2010,22:05:00,1.70261,1.70261,1.70199,1.70199,1.70476,1.70489,1.70406,1.70406,11,2010-01-24 22:05:00,0
2010-01-24 22:10:00,01-24-2010,22:10:00,1.70199,1.70211,1.70127,1.70127,1.70406,1.70413,1.70402,1.70402,4,2010-01-24 22:10:00,0
2010-01-24 22:15:00,01-24-2010,22:15:00,1.70127,1.70177,1.70122,1.70177,1.70402,1.70443,1.70402,1.70433,10,2010-01-24 22:15:00,0


In [6]:
%%time
from librairies.dagfeaturingfx import *
df = joblib.load('EURUSD_m5')
missing_candle(df,'m5',1)

Nombre (shape) de bougies manquante comparé au nombre (shape) de bougie total (909, 61) (812947, 61)
Pourcentage de trous dans la base : 0.11181540740048246
                        Close  CloseAsk  CloseBid      High  HighAsk  HighBid  \
2010-01-11 00:20:00  1.446960   1.44714   1.44678  1.447670  1.44776  1.44758   
2010-01-11 02:20:00  1.447950   1.44811   1.44779  1.448100  1.44829  1.44791   
2010-01-18 20:45:00  1.438345   1.43848   1.43821  1.438555  1.43865  1.43846   
2010-01-29 04:50:00  1.393285   1.39335   1.39322  1.393430  1.39351  1.39335   
2010-02-12 16:10:00  1.361485   1.36156   1.36141  1.361510  1.36157  1.36145   
...                       ...       ...       ...       ...      ...      ...   
2021-03-07 20:05:00  1.191500   1.19190   1.19110  1.191800  1.19230  1.19130   
2021-03-07 20:30:00  1.191400   1.19190   1.19090  1.191675  1.19205  1.19130   
2021-03-07 20:40:00  1.191575   1.19205   1.19110  1.191575  1.19205  1.19110   
2021-03-07 21:45:00  1.191900   1

In [11]:
df[(df.index>='2021-03-07 19:00:00')&(df.index<='2021-03-07 22:00:00')]

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,High_weekly,Low_weekly,Close_weekly,Open_daily,High_daily,Low_daily,Close_daily,Signal,datation,TimeJump
2021-03-07 19:30:00,1.191500,1.19190,1.1911,1.191500,1.19190,1.19110,1.19150,1.19190,1.19110,1.191500,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 19:30:00,0
2021-03-07 19:40:00,1.191500,1.19190,1.1911,1.191500,1.19190,1.19110,1.19145,1.19180,1.19110,1.191450,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 19:40:00,1
2021-03-07 20:05:00,1.191500,1.19190,1.1911,1.191800,1.19230,1.19130,1.19150,1.19190,1.19110,1.191600,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 20:05:00,1
2021-03-07 20:30:00,1.191400,1.19190,1.1909,1.191675,1.19205,1.19130,1.19140,1.19190,1.19090,1.191595,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 20:30:00,1
2021-03-07 20:40:00,1.191575,1.19205,1.1911,1.191575,1.19205,1.19110,1.19150,1.19190,1.19110,1.191500,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 20:40:00,1
2021-03-07 21:45:00,1.191900,1.19240,1.1914,1.191900,1.19240,1.19140,1.19190,1.19240,1.19140,1.191900,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 21:45:00,1
2021-03-07 21:50:00,1.191750,1.19210,1.1914,1.191865,1.19233,1.19140,1.19175,1.19210,1.19140,1.191865,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 21:50:00,0
2021-03-07 21:55:00,1.191870,1.19204,1.1917,1.191925,1.19215,1.19170,1.19172,1.19204,1.19140,1.191755,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 21:55:00,0
2021-03-07 22:00:00,1.191870,1.19204,1.1917,1.191960,1.19210,1.19182,1.19184,1.19204,1.19164,1.191870,...,1.21134,1.189315,1.19262,1.1915,1.193205,1.1914,1.19262,0,2021-03-07 22:00:00,0
